In [1]:
pip install datasets sacremoses transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Fou

In [1]:
from datasets import load_dataset
dataset = load_dataset("cfilt/iitb-english-hindi")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, GenerationConfig, MarianTokenizer

tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-hi")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-hi")

In [19]:
# Define a function to tokenize the data
def preprocess_function(examples):
    # Tokenize the input and target sentences
    inputs = tokenizer([ex['en'] for ex in examples["translation"]], max_length=128, truncation=True, padding=True)
    targets = tokenizer([ex['hi'] for ex in examples["translation"]], max_length=128, truncation=True, padding=True)
    # Add the tokenized inputs and targets to the dataset
    inputs['labels'] = targets['input_ids']
    return inputs

# Preprocess the train and validation datasets
#tokenized_datasets = dataset.map(preprocess_function, batched=True)
def get_fraction(dataset, fraction=0.1):
    # Calculate the size of the subset
    split_size = int(len(dataset) * fraction)
    # Select the first `split_size` examples
    return dataset.select(range(split_size))

# Get 10% of the train and validation datasets
fraction_train_dataset = get_fraction(dataset['train'], fraction=0.0001)
fraction_validation_dataset = get_fraction(dataset['validation'], fraction=0.0001)


In [20]:
# Apply the split
train_dataset_half = fraction_train_dataset.map(preprocess_function, batched=True)
validation_dataset_half = fraction_validation_dataset.map(preprocess_function, batched=True)

In [21]:
type(train_dataset_half['input_ids'][0][0])

int

In [25]:
from transformers import Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)  # Ensure model is the same as used for tokenization

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",          # Output directory where the model checkpoints will be saved
    eval_strategy="epoch",     # Evaluation strategy to use
    learning_rate=2e-5,              # Learning rate
    per_device_train_batch_size=4,   # Batch size for training
    per_device_eval_batch_size=4,    # Batch size for evaluation
    num_train_epochs=1,              # Number of training epochs
    weight_decay=0.01,               # Weight decay
    save_total_limit=1,              # Limit the number of checkpoints
    fp16=True,
    gradient_accumulation_steps=4,
    remove_unused_columns=False,
    predict_with_generate=True,
)


In [27]:
from transformers import Seq2SeqTrainer
# Initialize Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_half,
    eval_dataset=validation_dataset_half,
    tokenizer=tokenizer,
    data_collator=data_collator,

)


In [28]:
trainer.train()
model.save_pretrained("fine-tuned-model")
tokenizer.save_pretrained("fine-tuned-model")


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`translation` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [ ]:
src_text = [
    "this is not working"
]

translated = model.generate(**tokenizer(src_text, return_tensors="pt", padding=True))
[tokenizer.decode(t, skip_special_tokens=True) for t in translated]
